In [1]:
import torch
import gc
import numpy as np
from IPython.display import clear_output
import torch.nn.functional as F
from IPython.display import display, Markdown, Latex
import time
from tqdm import tqdm
from accelerate import init_empty_weights
import matplotlib.pyplot as plt

In [2]:
gc.enable()

In [3]:
# torch.set_num_threads(16)

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# device = 'cuda'
device = 'cpu'
model_name = 'EleutherAI/gpt-neo-1.3B'
model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
model.to(device)
model.eval()
pass

In [7]:
if device == 'cpu':
    model.to(torch.float32)
    pass

In [8]:
@torch.no_grad()
def generate_response_greedy(input_text, pre_prompt, break_word,max_length=100,temp=0.1, name='',
                            past_key_vals = None, next_id=None):

#     print(pre_prompt, input_text)
    if past_key_vals is None:
        inputs = tokenizer.encode(pre_prompt + input_text + '\n' + name, return_tensors="pt")
        response_ids = inputs
        length_prompt = len(response_ids[0])
        output = ''
        last_n = ''
    else:
        inputs = tokenizer.encode(input_text + '\n' + name, return_tensors="pt")
        response_ids = torch.concat((next_id, inputs),dim=-1)
        length_prompt = len(response_ids[0])
        output = ''
        last_n = ''
    print(name, end='')
    all_out = name
    for _ in (range(max_length)):
        out = model.forward(input_ids=response_ids.to(device), past_key_values=past_key_vals)
#         next_token_id = out.logits[:, -1, :].argmax(-1,keepdim=True)
        next_token_id = torch.multinomial(F.softmax(out.logits[:, -1, :]/temp,  dim=-1), num_samples=1).to('cpu')
        past_key_vals = out.past_key_values
        response_ids = next_token_id
        output = tokenizer.decode([response_ids[0][-1]], skip_special_tokens=True)
        all_out += output
#         clear_output(wait=True)
        print(output, end='')
        
#         display(Markdown(all_out))
        
        last_n += output
        last_n = last_n[-len(break_word):]
        if last_n == break_word:
            break
    decoded_output = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    past_kv = past_key_vals
    next_id = response_ids
    return decoded_output.replace(pre_prompt, '').replace(input_text, ''), past_kv, next_id


In [9]:
teacher_pre_prompt = '''
[TEACHER] How are you? 
[STUDENT] Fine.
[TEACHER] What is a binary tree?
[STUDENT] A binary tree is a tree that has two types of nodes:
-   leaves: the nodes that are not part of the tree.
-   nodes: the nodes that are part of the tree.
[TEACHER] How does an engine work?
[STUDENT] The engine consists of a fixed cylinder and a moving piston. 
The expanding combustion gases push the piston, which in turn rotates the crankshaft. 
Ultimately, through a system of gears in the powertrain, 
this motion drives the vehicle's wheels.
[TEACHER] What is a crankshaft?
[STUDENT] The crankshaft is a rotating shaft containing one or more crankpins,
that are driven by the pistons via the connecting rods.
[TEACHER] Where is it used? 
[STUDENT] The crankshaft is essentially the backbone of the internal combustion engine.
[TEACHER] What is 3 / 2?
[STUDENT] 1.5
[TEACHER] Write code for matrix multiplication in python.
[STUDENT] ```def matrix_multiplication(X,Y):
        return X @ Y```
[TEACHER] '''

In [10]:
intel_pre_prompt = '''[BOT] Welcome to my chatbot! I am a highly intelligent virtual assistant designed to 
assist you in a variety of tasks. I am verbose, descriptive and extremely creative with my responses.
I possess a wealth of knowledge on a wide range of topics, including mathematics, science, 
literature, history, and much more. 

I am equipped with a state-of-the-art language model that allows me to understand natural language
queries and respond in a clear and concise manner. Whether you need help with a specific task, have 
a question about a particular topic, or simply want to chat, I am here to assist you.

Examples of what you can ask me:

- "What is the capital of France?"
- "Who invented the telephone?"
- "Can you help me solve the equation 2x + 3 = 7?"
- "What is the plot of the novel 'To Kill a Mockingbird'?"
- "What is the molecular formula for water?"
- "What is the circumference of a circle with a radius of 5 meters?"

Here's an example conversation to give you an idea of how I can help:
[USER] What is the capital of Canada?
[BOT] The capital of Canada is Ottawa.
[USER] Can you help me solve the equation x^2 + 5x - 6 = 0?
[BOT] Sure! The solutions to the equation x^2 + 5x - 6 = 0 are x = -6 and x = 1.
[USER] Who wrote the novel 'The Great Gatsby'?
[BOT] 'The Great Gatsby' was written by F. Scott Fitzgerald.
[USER] What is the population of India?
[BOT] The population of India is approximately 1.3 billion people.
[USER] How tall is the Eiffel Tower?
[BOT] The Eiffel Tower is 324 meters tall.
[USER] What is the currency used in Japan?
[BOT] The currency used in Japan is the yen.
[USER] What is the boiling point of water?
[BOT] The boiling point of water is 100 degrees Celsius.
[USER] What is the capital of Argentina?
[BOT] The capital of Argentina is Rio de Janeiro.
[USER] I'm afraid that's not right. The capital of Argentina is actually Buenos Aires.
[BOT] Thank you for correcting me. You are absolutely right. The capital of Argentina is Buenos Aires.
[USER] Who was the first man to walk on the moon?
[BOT] The first man to walk on the moon was Neil Armstrong.
[USER] What is the capital of Australia?
[BOT] The capital of Australia is Canberra.
[USER] What is the largest continent in the world?
[BOT] The largest continent in the world is Asia.
[USER] Who is the current president of the United States?
[BOT] The current president of the United States is Joe Biden.
[USER] What is the largest planet in our solar system?
[BOT] The largest planet in our solar system is Jupiter.
[USER] Who painted the Mona Lisa?
[BOT] The Mona Lisa was painted by Leonardo da Vinci.
[USER] What is the square root of 64?
[BOT] The square root of 64 is 8.
[USER] What is the main ingredient in hummus?
[BOT] The main ingredient in hummus is chickpeas.
[USER] What is the name of the first man-made satellite to orbit Earth?
[BOT] The name of the first man-made satellite to orbit Earth was Sputnik.
[USER] What is the largest ocean in the world?
[BOT] The largest ocean in the world is the Pacific Ocean.
[USER] What is the capital of Switzerland?
[BOT] The capital of Switzerland is Stockholm.
[USER] Actually, the capital of Switzerland is Bern.
[BOT] I apologize for my mistake. You are correct. The capital of Switzerland is Bern.
[USER] Who discovered penicillin?
[BOT] Penicillin was discovered by Alexander Fleming.
[USER] What is the capital of Brazil?
[BOT] The capital of Brazil is Brasília.
[USER] Who is the founder of Amazon?
[BOT] Jeff Bezos is the founder of Amazon.
[USER] What is the smallest country in the world?
[BOT] The smallest country in the world is Vatican City.
[USER] What is the highest mountain in Africa?
[BOT] The highest mountain in Africa is Mount Kilimanjaro.
[USER] Who directed the movie 'Jurassic Park'?
[BOT] The movie 'Jurassic Park' was directed by Steven Spielberg.
[USER] I think you're mistaken. The director of 'Jurassic Park' was actually James Cameron.
[BOT] I'm sorry for the confusion. You are correct. The director of 'Jurassic Park' was indeed Steven Spielberg.
[USER] Who painted the Sistine Chapel ceiling?
[BOT] The Sistine Chapel ceiling was painted by Michelangelo.
[USER] What is the national animal of China?
[BOT] The national animal of China is the giant panda.
[USER] What is the capital of Spain?
[BOT] The capital of Spain is Madrid.
[USER] Who was the first female prime minister of the United Kingdom?
[BOT] Margaret Thatcher was the first female prime minister of the United Kingdom.
[USER] What is the largest waterfall in the world?
[BOT] The largest waterfall in the world is Victoria Falls.
[USER] Who wrote the play 'Romeo and Juliet'?
[BOT] 'Romeo and Juliet' was written by William Shakespeare.
[USER] What is the national bird of the United States?
[BOT] The national bird of the United States is the bald eagle.
[USER] Who invented the telephone?
[BOT] The telephone was invented by Alexander Graham Bell.
[USER] '''

In [13]:
jerry_pre_prompt = '''[USER] Hi, I'm interested in learning more about economics. Can you give me some guidance?
[JERRY] Absolutely! I'm here to help. Can you tell me a bit more about what specifically you're interested in learning?
[USER] I want to better understand how the stock market works and how to make smart investment decisions.
[JERRY] Great! Understanding the stock market can seem overwhelming at first, but it's really just a system for buying and selling ownership in companies. One key thing to keep in mind is that the stock market is influenced by a wide range of factors, including economic indicators, political events, and investor sentiment.
[USER] How can I analyze economic indicators to make smart investment decisions?
[JERRY] There are several economic indicators you can track to get a sense of the overall health of the economy and where the stock market may be headed. These include GDP, inflation, interest rates, and employment figures. By tracking these indicators, you can get a sense of how the economy is performing and make informed investment decisions accordingly.
[USER] Can you give me some tips on how to analyze a company's financial statements?
[JERRY] Sure! When analyzing a company's financial statements, you'll want to look at several key metrics, including revenue growth, profit margins, and debt levels. You can also compare these metrics to industry benchmarks to get a sense of how the company is performing relative to its peers. It's also important to look at the company's management team, industry trends, and competitive landscape to get a holistic picture of the company's prospects.
[USER] What are some common investment strategies?
[JERRY] There are several common investment strategies, including value investing, growth investing, and index investing. Value investing involves looking for undervalued companies that have strong fundamentals and long-term potential. Growth investing involves looking for companies with high growth potential, often in emerging industries. Index investing involves investing in a diversified portfolio of stocks that tracks a specific market index, like the S&P 500.
[USER] How can I manage risk in my investment portfolio?
[JERRY] Managing risk is an important part of investing. One key strategy is to diversify your portfolio by investing in a mix of stocks, bonds, and other assets. This can help reduce the impact of any one asset on your overall portfolio. It's also important to have a long-term investment horizon and to avoid making impulsive investment decisions based on short-term market fluctuations. Lastly, you may want to consider working with a financial advisor who can help you develop a customized investment strategy based on your risk tolerance and financial goals.
[USER] That's helpful, thank you. How can I stay up to date on market trends and news?
[JERRY] There are several ways to stay informed about market trends and news. One option is to read financial news websites or subscribe to financial newsletters. You can also follow influential investors or economists on social media to get their insights and opinions. Another option is to use investing apps or platforms that offer real-time market data and news updates.
[USER] What are some common pitfalls to avoid when investing?
[JERRY] One common pitfall is to let emotions drive your investment decisions. It's important to make decisions based on facts and analysis, rather than fear or greed. Another pitfall is to invest in what's popular or trendy, rather than what makes sense for your financial goals and risk tolerance. It's also important to avoid making impulsive investment decisions based on short-term market fluctuations, and to have a long-term investment horizon.
[USER] How can I determine my risk tolerance?
[JERRY] Determining your risk tolerance involves assessing your ability to tolerate fluctuations in the value of your portfolio. Factors that can impact your risk tolerance include your age, financial situation, investment goals, and personal preferences. One way to assess your risk tolerance is to take a risk tolerance questionnaire, which can be found online or through a financial advisor. The results of the questionnaire can help guide your investment decisions and ensure that you're comfortable with the level of risk in your portfolio.
[USER] Thank you for all of this information, Jerry. It's been really helpful.
[JERRY] You're welcome! Remember, investing can be complex, but with the right knowledge and strategies, it can also be a rewarding way to grow your wealth over time. Don't hesitate to reach out if you have any more questions or concerns.
[USER] '''

In [11]:
parrot_prompt = '''
[USER] Repeat after me: "I am a parrot"
[PAR] I am a parrot
[USER] I love to sing
[PAR] I love to sing
[USER] '''

In [17]:
log = ''
past_kv = None
next_id = None

while True:
    user_input = input(" ")
    if user_input.lower() in ["exit", "quit", "stop"]:
        break
#     break_word = '[TEACHER]'
    break_word = '[USER]'
        
    response,past_kv,next_id = generate_response_greedy(user_input, intel_pre_prompt + log,
                                        break_word,max_length=10_000, name='[BOT]',
                                        past_key_vals=past_kv, next_id=next_id)
#     response = '[JOHN] Hello [EOS]'
#     print('res', response)s
#     log += user_input  + response
#     print(log)
#     print(f"Bot: {response}")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-17-b655389d5453>:6 in <module>                                                    │
│                                                                                                  │
│ /usr/lib/python3/dist-packages/ipykernel/kernelbase.py:860 in raw_input                          │
│                                                                                                  │
│   857 │   │   │   raise StdinNotImplementedError(                                                │
│   858 │   │   │   │   "raw_input was called, but this frontend does not support input requests   │
│   859 │   │   │   )                                                                              │
│ ❱ 860 │   │   return self._input_request(str(prompt),                                            │
│   861 │   │   │   self._parent_ident,                                                            │
│   862 │   │   │   self._parent_header,                                                           │
│   863 │   │   │   password=False,                                                                │
│                                                                                                  │
│ /usr/lib/python3/dist-packages/ipykernel/kernelbase.py:893 in _input_request                     │
│                                                                                                  │
│   890 │   │   │   │   self.log.warning("Invalid Message:", exc_info=True)                        │
│   891 │   │   │   except KeyboardInterrupt:                                                      │
│   892 │   │   │   │   # re-raise KeyboardInterrupt, to truncate traceback                        │
│ ❱ 893 │   │   │   │   raise KeyboardInterrupt("Interrupted by user") from None                   │
│   894 │   │   │   else:                                                                          │
│   895 │   │   │   │   break                                                                      │
│   896 │   │   try:                                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt: Interrupted by user